## Decision Trees Implementation
*Uses Information Gain as the measurement to be split.*


# **LOGIC AND WORKING**

This code implements a decision tree algorithm for classification. Let's go through it step by step:

1. The `entropy` function calculates the entropy of a given column in the dataset. Entropy measures the impurity or disorder in the column's values.
2. The `information_gain` function calculates the information gain when splitting the data based on a specific feature. It uses the entropy function to compute the entropy of the target variable before and after the split and returns the information gain.
3. The `divide_data` function splits the dataset into two parts based on a given feature and threshold value. It uses boolean masking to create two subsets of the data: one where the feature values are greater than the threshold and the other where they are less than or equal to the threshold
4. The `find_count` function counts the occurrences of each class in the target variable of the given dataset and returns a list of counts.
5. The `DecisionTree` class represents a node in the decision tree. It has attributes such as `left` and `right` for storing references to child nodes, `key` and `val` for storing the feature and threshold value used to split the data, `count` for storing the class counts, `max_depth` for specifying the maximum depth of the tree, `depth` for keeping track of the current depth, and `target` for storing the predicted class at a leaf node.
6. The `train` method of the `DecisionTree` class builds the decision tree recursively. It takes the training dataset (`X_train`) and a list of class names (`names`) as input.
7. In the `train` method, the information gains for each feature are computed using the `information_gain` function. The feature with the highest information gain is selected as the splitting criterion (`key`), and the mean value of that feature is used as the threshold (`val`).
8. The method prints information about the current node, such as the level, class counts, current entropy, and splitting decision if applicable.
9. The data is split into left and right subsets using the `divide_data` function.
10. If there is only one class present in the current node, it becomes a leaf node, and the most frequent class is assigned as the target class.
11. If the maximum depth is reached, the node becomes a leaf node, and the most frequent class is assigned as the target class.
12. If neither of the above conditions is met, the `train` method is called recursively for the left and right subsets, creating child nodes.
13. Finally, the target class is assigned based on the majority class in the current node.
14. An instance of the `DecisionTree` class (`dt`) is created, and the `train` method is called on the training data (`data_`) with the class names (`names`).


In summary, this code recursively builds a decision tree by finding the best feature to split the data, creating child nodes, and assigning the target class at each node based on majority voting.

# **Implementation**

In [35]:
import numpy as np
import pandas as pd
from sklearn import datasets

In [36]:
data = datasets.load_iris()

In [37]:
data

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [38]:
names = data['target_names']

In [39]:
X = data['data']
X = np.array(X, dtype = int)
y = data['target']

In [40]:
data_ = pd.DataFrame(X)
data_.columns = data['feature_names']
data_['Output'] = y

In [41]:
data['feature_names']

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [42]:
data_.isna().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
Output               0
dtype: int64

In [43]:
# Define Entropy and Information Gain

In [44]:
def entropy(col):
    counts = np.unique(col, return_counts=True)
    N = float(col.shape[0])
    ent = 0.0
    for ix in counts[1]:
        p = ix / N
        ent += (-1.0 * p * np.log2(p))
    return ent

In [45]:
def information_gain(x_data, key, val):
    left, right = divide_data(x_data, key, val)
    l = float(left.shape[0]) / x_data.shape[0]
    r = float(right.shape[0]) / x_data.shape[0]
    if left.shape[0] == 0 or right.shape[0] == 0:
        return -1000000
    i_gain = entropy(x_data.Output) - (l * entropy(left.Output) + r * entropy(right.Output))
    return i_gain


In [46]:
def divide_data(x_data, key, val):
    mask = x_data[key] > val
    x_right = x_data[mask]
    x_left = x_data[~mask]
    return x_left, x_right

In [47]:
def find_count(X_train):
    count = []
    count.append(X_train[X_train['Output'] == 0].shape[0])
    count.append(X_train[X_train['Output'] == 1].shape[0])
    count.append(X_train[X_train['Output'] == 2].shape[0])
    return count

In [48]:
class DecisionTree:
    def __init__(self, depth=0, max_depth=5):
        self.left = None
        self.right = None
        self.key = None
        self.val = None
        self.count = None
        self.max_depth = max_depth
        self.depth = depth
        self.target = None
    
    def train(self, X_train, names):
        features = ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
        info_gains = []
        
        for ix in features:
            i_gain = information_gain(X_train, ix, X_train[ix].mean())
            info_gains.append(i_gain)
            
        self.key = features[np.argmax(info_gains)]
        self.val = X_train[self.key].mean()
        print("Level", self.depth)
        self.count = find_count(X_train)
        cnt = sum(1 for count in self.count if count > 0)
        for i in range(len(self.count)):
            if self.count[i]:
                print("Count of", names[i], "=", self.count[i])
        print("Current entropy =", entropy(X_train.Output))
        
        if cnt != 1:
            print("Splitting on tree feature", self.key, "with information gain", np.argmax(info_gains))
        
        data_left, data_right = divide_data(X_train, self.key, self.val)
        data_left = data_left.reset_index(drop=True)
        data_right = data_right.reset_index(drop=True)
         
        if cnt == 1:
            if X_train.Output.mean() >= 1.5:
                self.target = names[2]
            elif X_train.Output.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Reached leaf node")
            print()
            print()
            return
        
        if self.depth >= self.max_depth:
            if X_train.Output.mean() >= 1.5:
                self.target = names[2]
            elif X_train.Output.mean() <= 0.5:
                self.target = names[0]
            else:
                self.target = names[1]
            print("Max depth reached")
            print()
            print()
            return
        
        print()
        print()
        
        self.left = DecisionTree(depth=self.depth + 1, max_depth=self.max_depth)
        self.left.train(data_left, names)
        
        self.right = DecisionTree(depth=self.depth + 1, max_depth=self.max_depth)
        self.right.train(data_right, names)
        
        if X_train.Output.mean() >= 1.5:
            self.target = names[2]
        elif X_train.Output.mean() <= 0.5:
            self.target = names[0]
        else:
            self.target = names[1]
        return

dt = DecisionTree()
dt.train(data_, names)


Level 0
Count of setosa = 50
Count of versicolor = 50
Count of virginica = 50
Current entropy = 1.584962500721156
Splitting on tree feature petal width (cm) with information gain 3


Level 1
Count of setosa = 50
Current entropy = 0.0
Reached leaf node


Level 1
Count of versicolor = 50
Count of virginica = 50
Current entropy = 1.0
Splitting on tree feature petal length (cm) with information gain 2


Level 2
Count of versicolor = 48
Count of virginica = 6
Current entropy = 0.5032583347756457
Splitting on tree feature petal width (cm) with information gain 3


Level 3
Count of versicolor = 48
Count of virginica = 5
Current entropy = 0.45079138835466503
Splitting on tree feature petal length (cm) with information gain 2


Level 4
Count of versicolor = 11
Current entropy = 0.0
Reached leaf node


Level 4
Count of versicolor = 37
Count of virginica = 5
Current entropy = 0.5266170655714281
Splitting on tree feature sepal length (cm) with information gain 0


Level 5
Count of versicolor = 15
